# Hälsodataanalys

### Irene Grisenti - 28/11/2025

Detta projekt innehåller en analys av hälsodata med fokus på blodtryck, rökvanor och relaterade faktorer.  
Analysen inkluderar beskrivande statistik, hypotesprövning, konfidensintervall, enkel linjär regression, PCA och multipel regression för att identifiera mönster och påverkande variabler.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from src.analyzer import *
from src.stats_tests import *

plt.rcParams["figure.figsize"] = (10,6)
plt.rcParams["figure.dpi"] = 120
np.set_printoptions(precision=3)
pd.set_option("display.precision", 3)
plt.style.use("ggplot")

## Förberedelser

I den första delen laddar jag in datasetet och genomför en initial utforskning. Målet är att förstå datastrukturen, kontrollera om det finns saknade värden eller dubbletter samt ta fram grundläggande beskrivande statistik.    

**Inledande observation**
- Datasetet innehåller 800 rader, 9 kolumner. Inga saknade värden eller dubbletter
- Datatyper: 
    - Numeriska kontinuerliga: height, weight, systolic_bp, cholesterol
    - Numeriska diskreta: id, age, disease
    - Kategoriska: sex, smoker
- Range och fördelning:
    - Ålder varierar mellan 18 och 90 år, med ett medelvärde på cirka 49 år
    - Längd, vikt, blodtryck och kolesterol har också sannolika värden

In [ ]:
# Läsning av datasetet
df = pd.read_csv("data/health_study_dataset.csv")

# Skapa ett HealthAnalyzer-objekt
analyzer = HealthAnalyzer(df)

# Utforska datasetet
analyzer.explore()

# Städning
analyzer.cleaning()

## Beskrivande analys

I detta steg utför jag en beskrivande analys av datasetet för att bättre förstå vår population och dess hälsomått.  
Jag tittar på centrala tendenser och visualiserar olika aspekter av datan för att identifiera mönster och relationer mellan variabler:

- Beräkning av medelvärde, median, min och max för age, weight, height, systolic_bp och cholesterol.
- Visualisering:
    - Population översikt: 
        - Åldersfördelning efter kön 
        - Andel rökare vs icke-rökare
    - Hälsomått:
        - BMI-fördelning
        - Kolesterolnivåer efter kön
    - Samband:
        - Ålder vs kolesterol 
        - Systoliskt blodtryck för personer med/utan sjukdom

**Insikter**
- Män och kvinnor är ganska jämnt representerade över åldersspektrumet och förekomsten av rökare är låg.
- Normal viktintervall med få extremvärden. Liknande kolesterolnivåer mellan kvinnor och män.
- Scatter-diagrammet visar att kolesterolnivån tenderar att öka med åldern. 
- Det systoliska blodtrycket är liknande mellan grupper med eller utan sjukdom, vilket tyder på att det systoliska blodtrycket ensam inte kan indikera en grupp med eller utan sjukdom.

In [ ]:
specific_stats = analyzer.compute_stats(["age","weight","height"])
print(specific_stats)

In [ ]:
analyzer.plot_population_overview("img/fig1")
analyzer.plot_health_metrics("img/fig2")
analyzer.plot_relationships("img/fig3")

## Simulering av sjukdomsförekomst

I detta steg simulerar jag sjukdomsförekomst i en population med samma prevalens som i vårt dataset för att se hur mycket slumpen kan påverka resultaten.  

**Resultat och tolkning**  
Skillnaden mellan den verkliga sjukdomsfrekvensen (5.87 %) och den simulerade sjukdomsfrekvensen (5.4 %) är liten och ligger inom den förväntade variationen.  
Detta innebär att skillnaden vi ser i det verkliga data kan förklaras av slumpvariation snarare än av någon faktisk underliggande effekt.

In [ ]:
analyzer.simulation_disease(n=1000, seed=42)

## Konfidensintervall för systoliskt blodtryck

I detta steg uppskattar jag 95% konfidensintervall för systoliskt blodtryck med två metoder:

1. **Normal Approximation**, som antar att medelvärdet för urvalet är ungefär normalt fördelat.
2. **Bootstrap**, en resampling-metod som inte gör antaganden om fördelningen.

Syftet är att:
- Visa osäkerheten i vårt uppskattade medelvärde.  
- Jämföra resultaten från en analytisk metod (normalapproximation) med en mer flexibel, data-driven metod (bootstrap).  
- Bedöma om normalapproximationen är rimlig för detta dataset.

**Resultat och tolkning**  
Båda metoderna ger mycket liknande resultat, vilket tyder på att normalapproximationen är rimlig och att samplingfördelningen för medelvärdet är nära normal.

In [ ]:
analyzer.plot_systolic_bp_confidence_intervals(seed=42)

## Hypotesprövning - Rökare har högre blodtryck än icke-rökare

I detta steg testar jag om rökare har högre systoliskt blodtryck än icke-rökare. Syftet är att bedöma om den observerade skillnaden i blodtryck är så stor att den inte rimligen kan förklaras av slumpen.  

**Hypoteser**  
H0: Rökare och icke-rökare har samma blodtryck *(mean_smokers = mean_non_smokers)*  
H1: Rökare har högre blodtryck än icke-rökare *(mean_smokers > mean_non_smokers)*

**Motivering och val av test**  
Tvåprovstest (Welch’s t-test):
- Data är kvantitativa och de två grupperna är oberoende.
- Welch’s t-test är lämpligt eftersom grupperna har olika storlekar.
- Ensidigt test (höger svans) används eftersom vi specifikt vill testa om rökare har högre blodtryck.
- Signifikansnivå α = 0,05, vilket är standard.

**Resultat och tolkning**  
Eftersom p-värdet (p ≈ 0.32) är större än 0,05 kan vi inte förkasta H0. Teststatistiken var låg (t ≈ 0.45), vilket bekräftar att den observerade skillnaden inte är statistiskt signifikant.  
Det finns inte tillräckligt med bevis för att dra slutsatsen att rökare har högre blodtryck än icke-rökare.

In [ ]:
# Parametrar
smokers = df.loc[df["smoker"] == "Yes", "systolic_bp"]
non_smokers = df.loc[df["smoker"] == "No", "systolic_bp"]

n_smokers, n_non_smokers = len(smokers), len(non_smokers)
smokers_mean, non_smokers_mean = smokers.mean(), non_smokers.mean()
sd_smokers, sd_non_smokers = smokers.std(), non_smokers.std()

diff_means = smokers_mean - non_smokers_mean

# Statisiska tester
ttest_smokers_nonsmokers = ttest_smokers_vs_non_smokers(smokers, non_smokers)
t_stat, p_val, mean_diff = ttest_smokers_nonsmokers
effect = effect_calculation(smokers, non_smokers)

summary1 = pd.DataFrame({
    "Metric": [
        "n (smokers)", "n (non-smokers)",
        "Mean smokers", "Mean non-smokers",
        "SD smokers", "SD non-smokers",
        "Mean difference",
        "T-statistic", "P-value"
    ],
    "Value": [
        n_smokers, n_non_smokers,
        f"{smokers_mean:.2f}", f"{non_smokers_mean:.2f}",
        f"{sd_smokers:.2f}", f"{sd_non_smokers:.2f}",
        f"{diff_means:.2f}",
        f"{t_stat:.3f}", f"{p_val:.4f}"
    ]
})

summary1

**Visualisering** 
- **Systoliskt blodtryck efter grupp rökare/icke rökare**: grafen visar att fördelningen av systoliskt blodtryck i de två grupperna är mycket lika. Både median, medelvärde och den övergripande formen överlappar starkt mellan grupperna. Detta är i linje med resultatet från t-testet, som visade att skillnaden inte är statistiskt signifikant.

In [ ]:
# Violin plot: systoliskt blodtryck efter grupp rökare/icke rökare
data = [smokers, non_smokers]

fig5, ax = plt.subplots()
vp = ax.violinplot(data, showmeans=True, showmedians=True)
for body in vp["bodies"]:
    body.set_facecolor("#988ED5")
vp["cmedians"].set_color("#7A68A6")
vp["cbars"].set_color("#348ABD")
vp["cmaxes"].set_color("#348ABD")
vp["cmins"].set_color("#348ABD")
ax.set_title("Systolic BP per group")
ax.set_ylabel("Systolic BP (mmHg)")
ax.set_xticks([1, 2])
ax.set_xticklabels(["Smokers", "Non smokers"])
fig5.savefig("img/fig5")
plt.show()

## Undersökning av hypotes­test säkerhet

I detta steg undersöker jag hur känsligt t-testet är för att upptäcka en verklig skillnad i blodtryck mellan rökare och icke-rökare.  
Poweranalysen kompletterar hypotesprövningen genom att svara på frågan:
**”Skulle vi ha kunnat upptäcka en effekt om den fanns?”**

Beräkningar: 
1. **Effektstorlek (Cohen’s d)**  
2. **Teoretisk power** baserat på statistiska antaganden  
3. **Simulerad power** via Monte Carlo-simulering  
4. **Nödvändig effektstorlek** för att uppnå 80% power  

**Resultat**  
Både teoretisk och simulerad styrka beräknades för att jämföra resultaten och kontrollera att simuleringen stämmer med de teoretiska förväntningarna.

Resultaten visar att den statistiska styrkan (power) är mycket låg, cirka **0,11**.  
Detta beror på att den observerade skillnaden i medelvärde (≈ **0,47 mmHg**) är mycket liten jämfört med variationen inom grupperna (SD ≈ **13 mmHg**).  

Låg power innebär inte bara att testet misslyckas med att hitta en signifikant skillnad utan också att **testet inte hade rimliga förutsättningar att lyckas**.  

En vidare poweranalys visar att det skulle krävas en medelvärdesskillnad på ungefär **2,55 mmHg** för att uppnå **80% power**.

Detta bekräftar att resultatet från hypotesprövningen (icke-signifikant skillnad) är helt i linje med de mycket små effekter som observerats.  

**Tolkning**  
I praktiken tyder detta på att rökstatus **inte är förknippad med någon kliniskt eller statistiskt meningsfull förändring av systoliskt blodtryck** i detta urval.

In [ ]:
# Power berakningar
mathematical_power = theoretical_power(effect["effect size"], n_smokers, n_non_smokers, alpha=0.05)
simulated_power = simulate_ttest_power(smokers_mean, non_smokers_mean, sd_smokers, sd_non_smokers, n_smokers, n_non_smokers, alpha=0.05, n_sim=5000, seed=42)
required_power = power_analysis(n_smokers, n_non_smokers, sd_smokers, sd_non_smokers, alpha=0.05, power_target=0.8)

summary2 = pd.DataFrame({
    "Metric": [
        "Cohen's d", "Pooled SD",
        "Theoretical power", "Simulated power",
        "Required mean diff", "Required d"
    ],
    "Value": [
        f"{effect["effect size"]:.3f}", f"{effect["pooled sd"]:.3f}",
        f"{mathematical_power:.3f}", f"{simulated_power["hits"]:.3f}",
        f"{required_power["required mean difference"]:.2f}",
        f"{required_power["required d"]:.3f}"
    ]
})

summary2

## Linjär regression - Prediktion av systoliskt blodtryck

I detta steg använder jag en enkel linjär regression för att undersöka sambandet mellan systoliskt blodtryck och ålder. 

**Resultat och tolkning**  
- Skärning (intercept): 122.685 mmHg  
- Koefficient för ålder: 0.536 mmHg/år   
- Förklarad varians (R²): 0.369  

Koefficienten visar att blodtrycket i genomsnitt ökar med cirka 0.536 mmHg per år.  
R²-värdet på 0.37 indikerar att modellen förklarar ungefär 37% av variationen i systoliskt blodtryck med hjälp av ålder. Det innebär att ålder är en viktig, men inte ensam, förklarande faktor.  

In [ ]:
# Predicera systoliskt blodtryck utifrån ålder och vikt
X = df[["age"]].to_numpy() 
y = df["systolic_bp"].to_numpy() 

# Fit model
model = LinearRegression()
model.fit(X, y)

intercept_hat = float(model.intercept_)
coef_age = float(model.coef_[0])

# Prediktioner och residualer
y_hat = model.predict(X)
residuals = y - y_hat

# Manual R^2
ss_tot = np.sum((y - y.mean())**2)
ss_res = np.sum(residuals**2)
r2 = 1 - ss_res / ss_tot

print(f"""
Skärning (intercept)                 : {intercept_hat: .3f}
Koefficient för ålder (mmHg/år)      : {coef_age: .3f}
R^2                                  : {r2: .3f}

Tolkning av koefficienter:
- +10 år => {10 * coef_age:.2f} mmHg högre systoliskt blodtryck (i snitt)
""")

**Visualisering**  
- **Ålder vs Systoliskt blodtryck**: varje punkt representerar en individ. Regressionslinjen visar den förväntade trenden mellan ålder och blodtryck.   
- **Residualplot**: visualiserar skillnaden mellan observerade värden och modellens prediktioner. En jämn och slumpmässig spridning runt noll-linjen tyder på att linjärmodellen är rimlig och att inget tydligt mönster finns i residualerna.

In [ ]:
fig6, (ax1, ax2) = plt.subplots(1, 2)
fig6.suptitle("Linear regression", fontsize=17)

# Scatter: Ålder vs Systoliskt blodtryck
ax1.scatter(X[:,0], y, alpha=0.6)
x_line = np.linspace(X.min(), X.max(), 100)
y_line = intercept_hat + coef_age * x_line
ax1.plot(x_line, y_line, color="black", linewidth=1)
ax1.set_xlabel("Age")
ax1.set_ylabel("Systolic BP")
ax1.set_title("Age vs Systolic bp")

# Scatter: Residualplot
ax2.scatter(y_hat, residuals, color="#7A68A6", alpha=0.6)
ax2.axhline(0, color="black", linewidth=1)
ax2.set_xlabel("Explained value (ŷ)")
ax2.set_ylabel("Residual (Y - ŷ)")
ax2.set_title("Residuals")
plt.tight_layout()
fig6.savefig("img/fig6")
plt.show()

## Multipel linjär regression

I detta steg undersöker jag vilka faktorer som mest påverkar systoliskt blodtryck i vårt dataset. Vi har olika möjliga prediktorer (ålder, vikt, längd, kolesterol) och vill förstå deras relativa bidrag samt jämföra effektstorlekar på ett standardiserat sätt.  

Därför valdes **multipel linjär regression** som metod. Multipel regression är lämplig när vi vill modellera sambandet mellan en kontinuerlig beroende variabel (här: systoliskt blodtryck) och flera oberoende variabler, samt kontrollera för deras ömsesidiga påverkan.

**Dataförbereddelse**  
Prediktorerna (ålder, vikt, längd, kolesterol) standardiserades med `StandardScaler` för att jämföra relativa effekter. 

**Resultat**
Standardiserade koefficienter:
- **Ålder** är den starkast påverkande faktorn. En ökning med en standardavvikelse förväntas öka systoliskt blodtryck med ~7.6 mmHg.  
- **Vikt** har en måttlig effekt (~2.2 mmHg per SD).  
- **Längd och kolesterol** har mycket små effekter (0.47 och 0.45).    

Modellen förklarar cirka **41% av variationen** i systoliskt blodtryck (R² ≈ 0.41).

**Slutsats**
Analysen visar att **ålder och vikt är de mest betydelsefulla faktorerna** för systoliskt blodtryck i denna dataset. Modellen förklarar cirka 41% av variationen i blodtrycket, vilket innebär att en stor del av variationen fortfarande inte förklaras av de inkluderade variablerna. Detta tyder på att ytterligare faktorer kan behöva undersökas för att bättre förstå variationen i systoliskt blodtryck.

In [ ]:
# Target
y_multiple = df["systolic_bp"]

# Standardisera prediktorer
scaler = StandardScaler()
X_multiple = scaler.fit_transform(df[["age", "weight", "height", "cholesterol"]])

# Fit model
regr = LinearRegression()
regr.fit(X_multiple, y_multiple)

# Koefficienter
intercept_hat_multiple = float(regr.intercept_)
coef_names = ["age", "weight", "height", "cholesterol"]
coefficients = regr.coef_

# Prediktioner och residualer
y_hat_multiple = regr.predict(X_multiple)
residuals_multiple = y_multiple - y_hat_multiple

# R²
from sklearn.metrics import r2_score
r2_multiple = r2_score(y_multiple, y_hat_multiple)

print("Standardized coefficients (effect per 1 SD increase):")
for name, coef in zip(coef_names, coefficients):
    print(f"{name:12}: {coef:.3f} mmHg per SD")
print(f"""
R²          : {r2_multiple:.3f}
      
Tolkning:
- Ålder har störst påverkan på systoliskt blodtryck, följt av vikt.
- Längd och kolesterol har relativt små effekter.
- Modellen förklarar cirka {r2_multiple*100:.2f}% av variationen i blodtrycket.""")

**Visualisering**
- **Standardiserade koefficienter** visar effekten av varje prediktor på blodtrycket per en standardavvikelse.  
- **Predikterat vs observerat blodtryck** visar hur väl modellen predicerar blodtrycket. Punkter nära den svarta streckad linjen indikerar bra passform, medan avvikelser visar där modellen inte träffar exakt.

In [ ]:
fig8, (ax1, ax2) = plt.subplots(1, 2)
fig8.suptitle("Multiple regression", fontsize=17)

# Bar chart: Standardiserade koefficienter
ax1.bar(coef_names, coefficients, color="#348ABD", alpha=0.7)
ax1.set_title("Standardized Coefficients")
ax1.set_xlabel("Predictors")
ax1.set_ylabel("Coefficient (mmHg per SD increase)")

# Scatter: predikterat vs observerat blodtryck
ax2.scatter(y_hat_multiple, y_multiple, alpha=0.6)
ax2.plot([min(y_multiple), max(y_multiple)], [min(y_multiple), max(y_multiple)], "k--", lw=1)
ax2.set_title("Predicted vs actual Systolic BP")
ax2.set_xlabel("Predicted Systolic BP (mmHg)")
ax2.set_ylabel("Actual Systolic BP (mmHg)")

fig8.savefig("img/fig8")
plt.tight_layout()
plt.show()

## PCA - Identifiering av mönster i data

I detta steg har jag använt Principal Component Analysis (PCA) för att reducera datans komplexitet och visualisera de huvudsakliga variationerna. 

**Resultat**  
- **PC1 (x-axeln, varians: 0.406)** fångar den största variationen i data och de variabler som påverkar PC1 mest är ålder(0.573), systoliskt blodtryck(0.569) och kolesterol(0.526). Punkter längre åt vänster eller höger avviker mer från genomsnittet i dessa variabler.
  
- **PC2 (y-axeln, varians: 0.260)** fångar främst variation kopplad till kroppsstorlek, med starka bidrag från längd (0.694) och vikt (0.657). 

**Tolkning**
PCA-resultatet visar att ålder, blodtryck och kolesterol förklarar den största delen av variationen i datasetet. Längd och vikt står för den näst största delen av variationen. PCA hjälper därmed till att identifiera vilka variabler som är mest dominerande för att förklara skillnader mellan individer och ger en tydlig visuell överblick över hur data varierar längs två huvudsakliga dimensioner.

In [ ]:
# Välja numeriska variabler
X = df[["age", "height", "weight", "systolic_bp", "cholesterol"]]

# Skalera variablerna
X_scaled = StandardScaler().fit_transform(X)

# Utför PCA
pca = PCA(n_components=2)
principal_components = pca.fit_transform(X_scaled)

# Förklarad varians
explained_variance = pca.explained_variance_ratio_

print("Selected numeric variables: age, height, weight, systolic_bp, cholesterol")
for i, (component, var_ratio) in enumerate(zip(pca.components_, explained_variance), 1):
    print(f"PC{i} (variance: {var_ratio:.3f}): {component}")

**Visualisering**  
- **PCA komponenter**: grafen visar datapunkterna projicerade på PC1 och PC2. Punkterna bildar en relativt kompakt, ellipsliknande klunga utan tydliga separata grupper.
Spridningen längs PC1 representerar skillnader i ålder, blodtryck och kolesterol, medan spridningen längs PC2 representerar variation i kroppsstorlek. Punkter längre från mitten av figuren motsvarar individer som avviker mer från genomsnittet i de variabler som driver respektive komponent.

In [ ]:
# Scatter: PCA komponenter
fig7, ax = plt.subplots()
ax.scatter(principal_components[:,0], principal_components[:,1],
            color="#348ABD", alpha=0.6)
ax.set_title("PCA of health data")
ax.set_xlabel("PC1")
ax.set_ylabel("PC2")
plt.tight_layout()
fig7.savefig("img/fig7")
plt.show()

## Källor  

**Konfidensintervall**:  
- https://www.youtube.com/watch?v=xjYEYBvPaSc [13/11/2025]  

**ttest_ind**:  
- https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html [12/11/2025]

**solve_power**:  
- https://docs.w3cub.com/statsmodels/generated/statsmodels.stats.power.ttestindpower.solve_power [12/11/2025]  
- https://www.statsmodels.org/stable/generated/statsmodels.stats.power.TTestPower.solve_power.html#statsmodels.stats.power.TTestPower.solve_power [12/11/2025]

**Docstrings**:  
- https://numpydoc.readthedocs.io/en/latest/format.html [24/11/2025]

**Linjär Regression**:  
- https://www.youtube.com/watch?v=ZsJ-DbKpD3s&t=4036s [24/11/2025]

**PCA**:  
- https://www.geeksforgeeks.org/data-analysis/principal-component-analysis-pca/ [24/11/2025]
- https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html [24/11/2025]

**Multipel Regression**:
- https://www.w3schools.com/python/python_ml_multiple_regression.asp [24/11/2025]
- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html [24/11/2025]
- *Scaling*: 
    - https://www.w3schools.com/python/python_ml_scale.asp [24/11/2025]
    - https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html [24/11/2025]